In [1]:
#Chunk that sloads necessary packages & sets working environment to where the jupyterlab notebook file is 
%load_ext autoreload
%autoreload 2
import gnssvod as gv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pdb
import importlib
import zipfile
import os
import xarray as xr
import glob
import shutil
from gnssvod.hemistats.hemistats import hemibuild
#import georinex as gr
#import qgrid as interactive table 
from matplotlib.collections import PatchCollection
import matplotlib.dates as mdates
import shutil
from matplotlib import rcParams
from datetime import datetime, timedelta

In [2]:
# zentra data
zentra_file= '/Volumes/coraldelmar/zentra/coffe_zentra.csv'
df_zentra=pd.read_csv(zentra_file)

In [3]:
print(df_zentra.head())

        z6-21164                 Port 2              Port 2.1  \
0  Records: 4851               TEROS 21              TEROS 21   
1      Timestamp   kPa Matric Potential   °C Soil Temperature   
2   7/18/23 9:00                   -0.1                    25   
3  7/18/23 10:00                   -0.1                    25   
4  7/18/23 11:00                   -0.1                    25   

                  Port 5              Port 5.1             Port 6  \
0               TEROS 21              TEROS 21          PHYTOS 31   
1   kPa Matric Potential   °C Soil Temperature   min Leaf Wetness   
2                   -0.1                  24.7                 24   
3                   -0.1                  24.7                 53   
4                   -0.1                  24.7                  0   

                   Port 6.1        Port 6.2             Port 7  \
0                 PHYTOS 31       PHYTOS 31            Battery   
1   min Leaf Wetness (high)   Wetness Level  % Battery Percent

In [4]:
print(df_zentra.columns)

Index(['z6-21164', 'Port 2', 'Port 2.1', 'Port 5', 'Port 5.1', 'Port 6',
       'Port 6.1', 'Port 6.2', 'Port 7', 'Port 7.1', 'Port 8', 'Port 8.1'],
      dtype='object')


In [5]:
df_zentra=df_zentra.rename(columns={'z6-21164':'Timestamp',
                                    'Port 2':'Teros1',
                                    'Port 2.1':'Teros1 Temp',
                                    'Port 5': 'Teros2',
                                    'Port 5.1': 'Teros2 Temp',
                                    'Port 6': 'PHYTOS min',
                                    'Port 6.1': 'PHYTOS high',
                                    'Port 6.2':'Wetness Lvl',
                                    'Port 7': 'Battery PRctng',
                                    'Port 7.1': 'mV battery',
                                    'Port 8': 'barometer',
                                    'Port 8.1': 'loggerT'})
print(df_zentra.columns)

Index(['Timestamp', 'Teros1', 'Teros1 Temp', 'Teros2', 'Teros2 Temp',
       'PHYTOS min', 'PHYTOS high', 'Wetness Lvl', 'Battery PRctng',
       'mV battery', 'barometer', 'loggerT'],
      dtype='object')


In [6]:
#dropping unwanted columns & rows
columns2drop=['Battery PRctng','mV battery','barometer','loggerT']
df_zentra=df_zentra.drop(columns=columns2drop, errors='ignore')
df_zentra=df_zentra.drop(index=[0,1])                   

In [7]:
print(df_zentra.head(10))  

        Timestamp Teros1 Teros1 Temp Teros2 Teros2 Temp PHYTOS min  \
2    7/18/23 9:00   -0.1          25   -0.1        24.7         24   
3   7/18/23 10:00   -0.1          25   -0.1        24.7         53   
4   7/18/23 11:00   -0.1          25   -0.1        24.7          0   
5   7/18/23 12:00   -0.1          25   -0.1        24.7          0   
6   7/18/23 13:00   -0.1          25   -0.1        24.7          0   
7   7/18/23 14:00   -0.1          25   -0.1        24.7          0   
8   7/18/23 15:00   -0.1          25   -0.1        24.8          8   
9   7/18/23 16:00   -0.1        25.1   -0.1        24.8          0   
10  7/18/23 17:00   -0.1        25.1   -0.1        24.8          0   
11  7/18/23 18:00   -0.1        25.1   -0.1        24.8          0   

   PHYTOS high Wetness Lvl  
2            0         453  
3            0         448  
4            0         449  
5            0         448  
6            0         449  
7            0         448  
8            0         448

In [8]:
#soilvue data
# Filepaths for each year
soilvue50cm_files = [
    '/Volumes/coraldelmar/soilvue/2022/50 cm probe data 2022.csv',
    '/Volumes/coraldelmar/soilvue/2023/50 cm probe data 2023.csv',
    '/Volumes/coraldelmar/soilvue/2024/50 cm probe data 2024.csv',
    '/Volumes/coraldelmar/soilvue/2025/50 cm probe data 2025.csv'
]

soilvue1m_files = [
    '/Volumes/coraldelmar/soilvue/2023/100 cm probe data 2023.csv',
    '/Volumes/coraldelmar/soilvue/2024/100 cm probe data 2024.csv',
    '/Volumes/coraldelmar/soilvue/2025/100 cm probe data 2025.csv'
]

# Read and combine each probe depth
df_soilvue50cm = pd.concat([pd.read_csv(f) for f in soilvue50cm_files], ignore_index=True)
df_soilvue1m   = pd.concat([pd.read_csv(f) for f in soilvue1m_files], ignore_index=True)


print("50 cm shape:", df_soilvue50cm.shape)
print("1 m shape:", df_soilvue1m.shape)


50 cm shape: (688067, 7)
1 m shape: (659416, 7)


In [9]:
print(df_soilvue50cm.columns)

Index(['VWC', 'depth', 'File', 'Site', 'Temperature', 'DT', 'Probe_Name'], dtype='object')


In [10]:
columnsdrop=['File','Probe_Name']
df_soilvue50cm=df_soilvue50cm.drop(columns=columnsdrop)
df_soilvue50cm=df_soilvue50cm.rename(columns={'Site':'Plot',
                                             'DT':'Timestamp'})
print(df_soilvue50cm.head(10))

     VWC  depth     Plot  Temperature            Timestamp
0  0.007      5  Coffe 1        0.825  2022-09-02 17:00:00
1  0.006     10  Coffe 1        0.823  2022-09-02 17:00:00
2  0.015     20  Coffe 1        0.807  2022-09-02 17:00:00
3  0.016     30  Coffe 1        0.799  2022-09-02 17:00:00
4  0.015     40  Coffe 1        0.798  2022-09-02 17:00:00
5  0.014     50  Coffe 1        0.794  2022-09-02 17:00:00
6  0.222      5  Coffe 1       24.640  2022-09-02 18:00:00
7  0.190     10  Coffe 1       24.620  2022-09-02 18:00:00
8  0.438     20  Coffe 1       24.210  2022-09-02 18:00:00
9  0.466     30  Coffe 1       23.970  2022-09-02 18:00:00


In [11]:
print(df_soilvue1m)

          VWC  depth                                               File  \
0       0.107      5  5_8_2023/Coffe 2/Coffe 2_SoilVUE1m_hr_Coffee2.csv   
1       0.127     10  5_8_2023/Coffe 2/Coffe 2_SoilVUE1m_hr_Coffee2.csv   
2       0.069     20  5_8_2023/Coffe 2/Coffe 2_SoilVUE1m_hr_Coffee2.csv   
3       0.206     30  5_8_2023/Coffe 2/Coffe 2_SoilVUE1m_hr_Coffee2.csv   
4       0.225     40  5_8_2023/Coffe 2/Coffe 2_SoilVUE1m_hr_Coffee2.csv   
...       ...    ...                                                ...   
659411  0.125     40  1_7_2024/LFDP0204/LFDP0204_SVUE1m_hr_LFDP0204.csv   
659412  0.219     50  1_7_2024/LFDP0204/LFDP0204_SVUE1m_hr_LFDP0204.csv   
659413  0.289     60  1_7_2024/LFDP0204/LFDP0204_SVUE1m_hr_LFDP0204.csv   
659414  0.498     75  1_7_2024/LFDP0204/LFDP0204_SVUE1m_hr_LFDP0204.csv   
659415  0.633    100  1_7_2024/LFDP0204/LFDP0204_SVUE1m_hr_LFDP0204.csv   

            Site  Temperature                   DT  \
0        Coffe 2        24.35  2023-05-05 12:

In [12]:
columnsdrop=['File','Probe_Name']
df_soilvue1m=df_soilvue1m.drop(columns=columnsdrop)
df_soilvue1m=df_soilvue1m.rename(columns={'Site':'Plot',
                                         'DT':'Timestamp'})
print(df_soilvue1m.head(10))

     VWC  depth     Plot  Temperature            Timestamp
0  0.107      5  Coffe 2        24.35  2023-05-05 12:00:00
1  0.127     10  Coffe 2        24.01  2023-05-05 12:00:00
2  0.069     20  Coffe 2        23.55  2023-05-05 12:00:00
3  0.206     30  Coffe 2        23.40  2023-05-05 12:00:00
4  0.225     40  Coffe 2        23.25  2023-05-05 12:00:00
5  0.051     50  Coffe 2        23.08  2023-05-05 12:00:00
6  0.110     60  Coffe 2        22.87  2023-05-05 12:00:00
7  0.210     75  Coffe 2        22.66  2023-05-05 12:00:00
8  0.531    100  Coffe 2        22.42  2023-05-05 12:00:00
9  0.112      5  Coffe 2        23.91  2023-05-05 13:00:00


In [13]:
#addressing the mixed/missing timestamps
df_soilvue1m['Timestamp'] = pd.to_datetime(df_soilvue1m['Timestamp'], format='mixed', errors='coerce')

In [14]:
df_soilvue1m['Timestamp'] = df_soilvue1m['Timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
df_soilvue1m

,VWC,depth,Plot,Temperature,Timestamp
0,0.107,5,Coffe 2,24.35,2023-05-05 12:00:00
1,0.127,10,Coffe 2,24.01,2023-05-05 12:00:00
2,0.069,20,Coffe 2,23.55,2023-05-05 12:00:00
3,0.206,30,Coffe 2,23.40,2023-05-05 12:00:00
4,0.225,40,Coffe 2,23.25,2023-05-05 12:00:00
...,...,...,...,...,...
659411,0.125,40,LFDP0204,22.71,2025-01-07 10:00:00
659412,0.219,50,LFDP0204,22.85,2025-01-07 10:00:00
659413,0.289,60,LFDP0204,22.97,2025-01-07 10:00:00
659414,0.498,75,LFDP0204,23.13,2025-01-07 10:00:00


In [15]:
#creating dfs per plot
df = df_soilvue1m.copy()

# Standardize Plot names
df["Plot"] = df["Plot"].astype(str).str.strip()
df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")

# Map original Plot names to short variable-style names
plot_map = {
    "Omar Upper": "omaru",
    "Coffe 1": "coffe1",
    "Coffe 2": "coffe2",
    "Omar Lower": "omarl",
    "LFDP090102": "grid02",
    "LFDP0204": "grid04"
}

# Generate a dict first (safe)
dfs_by_plot = {plot_map[p]: g.sort_values(["Timestamp", "depth"]).reset_index(drop=True)
               for p, g in df.groupby("Plot") if p in plot_map}

# Turn them into real variables in your session
for shortname, g in dfs_by_plot.items():
    globals()[f"{shortname}_1m"] = g

# Example: use directly
print(omaru_1m.head())
print(omarl_1m.head())
print(grid04_1m.head())
print(grid02_1m.head())
print(coffe1_1m.head())
print(coffe2_1m.head())

     VWC  depth        Plot  Temperature           Timestamp
0  0.101      5  Omar Upper        23.31 2023-05-09 12:00:00
1  0.000     10  Omar Upper        22.49 2023-05-09 12:00:00
2  0.145     20  Omar Upper        22.26 2023-05-09 12:00:00
3  0.287     30  Omar Upper        22.26 2023-05-09 12:00:00
4  0.494     40  Omar Upper        22.26 2023-05-09 12:00:00
     VWC  depth        Plot  Temperature           Timestamp
0  0.044      5  Omar Lower        23.56 2023-05-05 16:00:00
1  0.000     10  Omar Lower        23.30 2023-05-05 16:00:00
2  0.046     20  Omar Lower        22.91 2023-05-05 16:00:00
3  0.223     30  Omar Lower        22.86 2023-05-05 16:00:00
4  0.139     40  Omar Lower        22.78 2023-05-05 16:00:00
     VWC  depth      Plot  Temperature           Timestamp
0  0.187      5  LFDP0204        24.17 2023-05-05 17:00:00
1  0.415     10  LFDP0204        23.95 2023-05-05 17:00:00
2  0.279     20  LFDP0204        23.66 2023-05-05 17:00:00
3  0.375     30  LFDP0204       

NameError: name 'grid02_1m' is not defined

In [16]:
#creating dfs per plot
df50 = df_soilvue50cm.copy()

# Standardize Plot names
df50["Plot"] = df50["Plot"].astype(str).str.strip()
df50["Timestamp"] = pd.to_datetime(df50["Timestamp"], errors="coerce")

# Map original Plot names to short variable-style names
plot_map = {
    "Omar Upper": "omaru",
    "Coffe 1": "coffe1",
    "Coffe 2": "coffe2",
    "Omar Lower": "omarl",
    "LFDP090102": "grid02",
    "LFDP0204": "grid04"
}

# Generate a dict first (safe)
dfs_by_plot = {plot_map[p]: g.sort_values(["Timestamp", "depth"]).reset_index(drop=True)
               for p, g in df50.groupby("Plot") if p in plot_map}

# Turn them into real variables in your session
for shortname, g in dfs_by_plot.items():
    globals()[f"{shortname}_50cm"] = g

# Example: use directly
print(omaru_50cm.head())
print(omarl_50cm.head())
print(grid04_50cm.head())
print(grid02_50cm.head())
print(coffe1_50cm.head())
print(coffe2_50cm.head())

     VWC  depth        Plot  Temperature           Timestamp
0  0.128      5  Omar Upper        24.27 2022-09-02 14:00:00
1  0.024     10  Omar Upper        24.12 2022-09-02 14:00:00
2  0.450     20  Omar Upper        23.96 2022-09-02 14:00:00
3  0.549     30  Omar Upper        24.00 2022-09-02 14:00:00
4  0.477     40  Omar Upper        23.94 2022-09-02 14:00:00
     VWC  depth        Plot  Temperature           Timestamp
0  0.000      5  Omar Lower        25.73 2022-09-02 13:00:00
1  0.019     10  Omar Lower        25.16 2022-09-02 13:00:00
2  0.456     20  Omar Lower        25.03 2022-09-02 13:00:00
3  0.519     30  Omar Lower        25.16 2022-09-02 13:00:00
4  0.501     40  Omar Lower        25.09 2022-09-02 13:00:00
     VWC  depth      Plot  Temperature           Timestamp
0  0.168      5  LFDP0204        24.97 2022-09-02 17:00:00
1  0.000     10  LFDP0204        24.88 2022-09-02 17:00:00
2  0.181     20  LFDP0204        24.60 2022-09-02 17:00:00
3  0.483     30  LFDP0204       

In [ ]:
# weather data
weather_file='/Volumes/coraldelmar/weather.station_alonso/2024/2024-1h.xlsx'
df_weather=pd.read_excel(weather_file)
print(df_weather)

In [ ]:
print(df_weather.columns)

In [ ]:
drop=['Hour','RECORD','BattV_Avg','PTemp_C_Avg','WindDir','WS_ms_Avg','NR_Wm2_Avg','Unnamed: 16']
df_weather=df_weather.drop(columns=drop,errors='ignore')

In [ ]:
df_weather=df_weather.rename(columns={'TIMESTAMP': 'Timestamp'})

In [ ]:
print(df_weather.head(10))

In [ ]:
# band dendrometer
bandDendro_file='/Volumes/coraldelmar/band.dendrometer/BandDendrometers_Census5_May2024.xlsx'
df_bandDendro=pd.read_excel(bandDendro_file)
print(df_bandDendro.head())

In [ ]:
print(df_bandDendro.columns)

In [ ]:
dropz=['Note']
df_bandDendro=df_bandDendro.drop(columns=dropz,errors='ignore')

In [ ]:
df_bandDendro =df_bandDendro.replace("-", pd.NA).dropna().reset_index(drop=True)
print(df_bandDendro.head(20))

In [ ]:
print(df_bandDendro.columns)

In [ ]:

df_bandDendro["Timestamp"] = pd.to_datetime(df_bandDendro["Date"].astype(str) + " " + df_bandDendro["Time"].astype(str))

df_bandDendro = df_bandDendro.drop(columns=["Date", "Time"])

# Check the result
print(df_bandDendro.head())


In [ ]:
# #as strings in order to drop
# df_bandDendro['Timestamp'] = pd.to_datetime(
#     df_bandDendro['Date'].astype(str) + ' ' + df_bandDendro['Time'].astype(str)
# )

# # drop 'Date' and 'Time' columns
# df_bandDendro = df_bandDendro.drop(columns=['Date', 'Time'])

# print(df_bandDendro.head())



In [ ]:
# loading combined sapflow data from loading_sapflow.ipynb
# march 2024 data
march2025_sapflow='/Volumes/coraldelmar/sapflow/2025/marzo2025/combined_sapflow_data.csv'
df_sapflow0325=pd.read_csv(march2025_sapflow)
print(df_sapflow0325)


In [ ]:
#removing commas
#df_sapflow0324 = df_sapflow0324.applymap(lambda x: x.replace(',', '') if isinstance(x, str) else x)


In [ ]:
df_sapflow0325

In [ ]:
print(df_sapflow0325.columns)

In [ ]:
drop3=['DOY','Sapwood dry weight (g)','Sapwood fresh volume (cm^3)', 'Probe spacing (cm)',
       'Xylem radius (cm)',' Thermistor distance (cm)',' First thermistor depth (cm)',
       'Sapwood depth (cm)','Beyond last thermistor','Advanced thermal diffusivity',
       'Sapwood fresh weight (g)','Thermistor distance (cm)','First thermistor depth (cm)',
       'Bark thickness (cm)','Thermal diffusivity (cm^2.s^-1)',
       'Wound diameter (cm)','Wound correction factor (-)','Vs correction factor (-)','Sensor properties:',
      'Calculation coefficients:']
df_sapflow0325=df_sapflow0325.drop(columns=drop3,errors='ignore')
df_sapflow0325=df_sapflow0325.rename(columns={'Date Time': 'Timestamp'})
df_sapflow0325.head(10)

In [ ]:
print(df_sapflow0325.columns)

In [ ]:
# #consistent date time
# #
# df_sapflow0324.columns = df_sapflow0324.columns.str.strip()

# #datetime
# df_sapflow0324['Timestamp'] = pd.to_datetime(
#     df_sapflow0324['Timestamp'], format='%d/%m/%Y %H:%M', errors='coerce'
# )

# #droping invalid timestamps
# df_sapflow0324 = df_sapflow0324.dropna(subset=['Timestamp'])

# # dropping any duplicate timestamps
# #df_sapflow0324 = df_sapflow0324.drop_duplicates(subset='Timestamp')

# # sort by timestamps
# df_sapflow0324 = df_sapflow0324.sort_values('Timestamp')
# '''
# # hourly timestamp range
# full_range = pd.date_range(
#     start=df_sapflow0324['Timestamp'].min(),
#     end=df_sapflow0324['Timestamp'].max(),
#     freq='h'  
# )

# # reindex timestamp
# df_sapflow0324 = df_sapflow0324.set_index('Timestamp').reindex(full_range)
# '''
# # index back to column 
# df_sapflow0324.index.name = 'Timestamp'
# #df_sapflow0324 = df_sapflow0324.reset_index()

# # 
# print(df_sapflow0324.head())


In [ ]:
print(df_sapflow0325['SpeciesCode'].unique())


In [ ]:
# loading combined sapflow data from loading_sapflow.ipynb
# may 2024 data
may2025_sapflow='/Volumes/coraldelmar/sapflow/2025/mayo2025/combined_sapflow_data.csv'
df_sapflow0525=pd.read_csv(may2025_sapflow)
print(df_sapflow0525)

In [ ]:
drop3=['DOY','Sapwood dry weight (g)','Sapwood fresh volume (cm^3)', 'Probe spacing (cm)',
       'Xylem radius (cm)',' Thermistor distance (cm)',' First thermistor depth (cm)',
       'Sapwood depth (cm)','Beyond last thermistor','Advanced thermal diffusivity',
       'Sapwood fresh weight (g)','Thermistor distance (cm)','First thermistor depth (cm)',
       'Bark thickness (cm)','Thermal diffusivity (cm^2.s^-1)',
       'Wound diameter (cm)','Wound correction factor (-)','Vs correction factor (-)','Sensor properties:',
      'Calculation coefficients:']
df_sapflow0525=df_sapflow0525.drop(columns=drop3,errors='ignore')
df_sapflow0525=df_sapflow0525.rename(columns={'Date Time': 'Timestamp'})
df_sapflow0525.head(10)

In [ ]:
# #consistent date time
# #
# df_sapflow0524.columns = df_sapflow0524.columns.str.strip()

# #datetime
# df_sapflow0524['Timestamp'] = pd.to_datetime(
#     df_sapflow0524['Timestamp'], format='%d/%m/%Y %H:%M', errors='coerce'
# )

# #droping invalid timestamps
# df_sapflow0524 = df_sapflow0524.dropna(subset=['Timestamp'])

# # dropping any duplicate timestamps
# df_sapflow0524 = df_sapflow0524.drop_duplicates(subset='Timestamp')

# # sort by timestamps
# df_sapflow0524 = df_sapflow0524.sort_values('Timestamp')

# # hourly timestamp range
# full_range = pd.date_range(
#     start=df_sapflow0524['Timestamp'].min(),
#     end=df_sapflow0524['Timestamp'].max(),
#     freq='h'  
# )

# # reindex timestamp
# df_sapflow0524 = df_sapflow0524.set_index('Timestamp').reindex(full_range)

# # index back to column 
# df_sapflow0524.index.name = 'Timestamp'
# df_sapflow0524 = df_sapflow0524.reset_index()

# # 
# print(df_sapflow0524.head())


In [ ]:
print(df_sapflow0525['SpeciesCode'].unique())
#no guagui nor manbid 

In [ ]:
df_sapflow0325.to_csv('/Volumes/coraldelmar/analyses/sapflow0325.csv')
df_sapflow0525.to_csv('/Volumes/coraldelmar/analyses/sapflow0525.csv')

In [ ]:
#saving as csvs
df_zentra.to_csv('/Volumes/coraldelmar/analyses/zentra.csv', index=False)
df_soilvue50cm.to_csv('/Volumes/coraldelmar/analyses/soilvue50cm.csv', index=False)
df_soilvue1m.to_csv('/Volumes/coraldelmar/analyses/soilvue1m.csv',index=False)
df_bandDendro.to_csv('/Volumes/coraldelmar/analyses/bandDendro.csv',index=False)
df_sapflow0325.to_csv('/Volumes/coraldelmar/analyses/sapflow0325.csv',index=False)
df_sapflow0525.to_csv('/Volumes/coraldelmar/analyses/sapflow0525.csv',index=False)

omaru_1m.to_csv('/Volumes/coraldelmar/analyses/omaru_1m.csv', index=False)
omarl_1m.to_csv('/Volumes/coraldelmar/analyses/omarl_1m.csv', index=False)
grid04_1m.to_csv('/Volumes/coraldelmar/analyses/grid04_1m.csv', index=False)
grid02_1m.to_csv('/Volumes/coraldelmar/analyses/grid02_1m.csv', index=False)
coffe1_1m.to_csv('/Volumes/coraldelmar/analyses/coffe1_1m.csv', index=False)
coffe2_1m.to_csv('/Volumes/coraldelmar/analyses/coffe2_1m.csv', index=False)
#
omaru_50cm.to_csv('/Volumes/coraldelmar/analyses/omaru_50cm.csv', index=False)
omarl_50cm.to_csv('/Volumes/coraldelmar/analyses/omarl_50cm.csv', index=False)
grid04_50cm.to_csv('/Volumes/coraldelmar/analyses/grid04_50cm.csv', index=False)
grid02_50cm.to_csv('/Volumes/coraldelmar/analyses/grid02_50cm.csv', index=False)
coffe1_50cm.to_csv('/Volumes/coraldelmar/analyses/coffe1_50cm.csv', index=False)
coffe2_50cm.to_csv('/Volumes/coraldelmar/analyses/coffe2_50cm.csv', index=False)